In [1]:
import pandas as pd
import numpy as np
import gensim
import collections
import smart_open
import random

In [8]:
df = pd.read_pickle('saved_df1.pkl')

In [9]:
df.dropna(subset=['body', 'summary', 'headline'], inplace=True)

In [10]:
len(df)

614158

In [11]:
def read_corpus(f, tokens_only=False):
    for i, line in enumerate(f):
        if tokens_only:
            yield gensim.utils.simple_preprocess(line)
        else:
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [17]:
df2 = df.sample(10000)

In [18]:
combined = [h + ' ' + s + ' ' + b for h, s, b in zip(list(df2['headline']), list(df2['summary']), list(df2['body']))]

In [19]:
len(combined)

10000

In [20]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(combined, test_size=0.1, random_state=42)

In [21]:
len(train), len(test)

(9000, 1000)

In [25]:
train_corpus = list(read_corpus(train))
test_corpus = list(read_corpus(test, tokens_only=True))

In [26]:
train_corpus[0]

TaggedDocument(words=['how', 'football', 'is', 'improving', 'the', 'lives', 'of', 'denmark', 'homeless', 'in', 'denmark', 'scientists', 'have', 'discovered', 'that', 'football', 'can', 'significantly', 'improve', 'the', 'health', 'of', 'homeless', 'and', 'other', 'socially', 'disadvantaged', 'groups', 'project', 'in', 'denmark', 'is', 'encouraging', 'homeless', 'people', 'to', 'get', 'off', 'the', 'streets', 'and', 'onto', 'the', 'football', 'pitch', 'with', 'dramatic', 'results', 'the', 'coin', 'is', 'tossed', 'in', 'just', 'few', 'seconds', 'the', 'game', 'kicks', 'off', 'while', 'running', 'up', 'and', 'down', 'the', 'field', 'trying', 'to', 'score', 'goal', 'and', 'defend', 'their', 'own', 'marco', 'glentvor', 'and', 'peter', 'fromm', 'sweat', 'for', 'their', 'health', 'they', 'say', 'football', 'has', 'changed', 'their', 'lives', 'in', 'denmark', 'scientists', 'have', 'found', 'that', 'football', 'can', 'significantly', 'improve', 'the', 'health', 'of', 'homeless', 'and', 'other',

In [27]:
test_corpus[0]

['paul',
 'scally',
 'gillingham',
 'chairman',
 'reveals',
 'new',
 'stadium',
 'plans',
 'gillingham',
 'chairman',
 'paul',
 'scally',
 'says',
 'new',
 'stadium',
 'could',
 'allow',
 'the',
 'club',
 'to',
 'sustain',
 'place',
 'in',
 'the',
 'top',
 'two',
 'divisions',
 'gillingham',
 'chairman',
 'paul',
 'scally',
 'says',
 'new',
 'stadium',
 'could',
 'allow',
 'the',
 'club',
 'to',
 'sustain',
 'place',
 'in',
 'the',
 'top',
 'two',
 'divisions',
 'of',
 'english',
 'football',
 'the',
 'league',
 'one',
 'side',
 'are',
 'hoping',
 'to',
 'construct',
 'new',
 'stadium',
 'at',
 'mill',
 'hill',
 'in',
 'medway',
 'opposite',
 'their',
 'training',
 'ground',
 'the',
 'gills',
 'have',
 'been',
 'based',
 'at',
 'their',
 'current',
 'seater',
 'priestfield',
 'ground',
 'in',
 'medway',
 'since',
 'when',
 'came',
 'here',
 'in',
 'realised',
 'that',
 'there',
 'was',
 'no',
 'long',
 'term',
 'future',
 'for',
 'this',
 'club',
 'on',
 'this',
 'site',
 'scally',
 's

In [28]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=400, min_count=2, epochs=15)

In [29]:
model.build_vocab(train_corpus)

In [30]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 2min 3s, sys: 1.72 s, total: 2min 5s
Wall time: 45 s


In [31]:
ranks = []
second_ranks = []

for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words) # Embeds document into vector space
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs)) # Checks how similar the inferred vector is to all other documents. 
    rank = [docid for docid, sim in sims].index(doc_id) # Checks which document the inferred vector is most similar to (should be itself i.e. the top 1 article)
    ranks.append(rank)
    
    second_ranks.append(sims[1]) #Second most similar article

/anaconda3/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [49]:
ranks

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [50]:
second_ranks

[(2887, 0.5470970273017883),
 (8897, 0.5608168840408325),
 (1581, 0.5642291307449341),
 (6809, 0.6331863403320312),
 (1641, 0.747954249382019),
 (8641, 0.5707781314849854),
 (7838, 0.6543934345245361),
 (7247, 0.6092836856842041),
 (6833, 0.6222963333129883),
 (4500, 0.628307580947876),
 (5787, 0.9263832569122314),
 (2088, 0.5685209035873413),
 (7648, 0.6190728545188904),
 (1294, 0.7144190669059753),
 (5591, 0.6136232614517212),
 (7755, 0.5560190081596375),
 (2046, 0.6229545474052429),
 (5696, 0.8069941401481628),
 (6205, 0.626198410987854),
 (3524, 0.665336549282074),
 (1854, 0.6255166530609131),
 (3788, 0.6270136833190918),
 (6090, 0.5684628486633301),
 (4090, 0.6147289276123047),
 (5626, 0.6398587822914124),
 (891, 0.5766580700874329),
 (4256, 0.6709205508232117),
 (4051, 0.5858308672904968),
 (6832, 0.7155099511146545),
 (8389, 0.5370103716850281),
 (2467, 0.6794776916503906),
 (2944, 0.7537570595741272),
 (5692, 0.5319092273712158),
 (5409, 0.6908259391784668),
 (5630, 0.509249567

In [51]:
from collections import Counter

Counter(ranks)

Counter({0: 8987, 6: 3, 1: 7, 2: 1, 3798: 1, 3: 1})

In [55]:
doc_id = 8999

print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))

print(u'SIMILAR/DISIMILAR DOCS PER MODEL %s:\n' % model)

for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))
    

Document (8999): «gary johnson yeovil town relegation must improve club gary johnson says yeovil must come back stronger after their relegation from the championship which is all but confirmed manager gary johnson says yeovil town must come back stronger from their relegation from the championship which was all but confirmed on monday the glovers defeat by huddersfield left them six points from safety with two matches left needing goal swing to stay up on goal difference won eight drawn lost with two games remaining highest position th goals scored conceded top scorer ishmael miller with average goals per game johnson who led the club to league one promotion last season told bbc somerset the club got to be bigger going back to league one means we can regroup and come back stronger johnson was in charge in when non league yeovil were promoted to the football league for the first time in their history ten years later following spells in charge at bristol city northampton and peterborough

In [57]:
#Pick a random document from the corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (4277): «radamel falcao unhappy striker could leave man utd says mother manchester united striker radamel falcao is unhappy at the club and could leave in the summer says his mother radamel falcao mother has said the striker is unhappy at manchester united and could leave in the summer the colombian joined united on loan from monaco in september but has scored just four goals in appearances falcao has only started one of united last eight premier league matches the loss at chelsea even if he unhappy not to be in the starting xi he has to accept the coach decisions carmenza zarate told colombian newspaper el espectador often remind him that after test or difficulty blessings arrive if god shows him new path he ll leave manchester if not he ll stay we hope for bigger and better things god hasn abandoned us and my son will become again what he has been goal scorer before his move to old trafford falcao was one of the most prolific strikers in the world with goals in games f